# Saving and exporting circuits

OpenDSS has a command to save the current circuit, e.g. `save circuit dir=some/path/`. 

During the development of the JSON functions for whole circuits (`Circuit.ToJSON()` and `Circuit.FromJSON()`) on DSS-Extensions, we took the oportunity to add some options to the save command, exposing it as a new function in API. This is thus avaiable in most projects under DSS-Extensions. Since many third-party software use OpenDSSDirect.py to inspect and export data, this is being added to the ODD.py documentation, but most of the information here applies to other Python packages and other projects in general on DSS-Extensions.

The options are controlled through a bitmask integer, with the options from [DSSSaveFlags](https://dss-extensions.org/OpenDSSDirect.py/enumerations.html#dss_python_backend.enums.DSSSaveFlags), as [documented in the API reference for `Circuit.Save()`](https://dss-extensions.org/OpenDSSDirect.py/opendssdirect.html#opendssdirect.Circuit.ICircuit.Save).

This notebook shows some examples of the options. 

If you have a suggestion for new flags, please feel free to suggest on GitHub, either through the [ODD.py issues](https://github.com/dss-extensions/OpenDSSDirect.py/issues), or general issues/discussions on  https://github.com/dss-extensions/dss-extensions

Remember that both the `save circuit` command and the function used here export snapshots of the circuit. If the original .DSS script is a complete simulation, if won't be tracked, currently.

Another limitation is that some file references are kept as-is. This is not typically an issue since most fields are read into numeric fields, which are exported OK.

**Notebook requirements**

In [ ]:
# When running via Colab, install the package first
import os, subprocess
if os.getenv("COLAB_RELEASE_TAG"):
    print(subprocess.check_output('pip install opendssdirect.py[extras]', shell=True).decode())

# Download the sample circuits and test cases if they do not exist already
from dss.examples import download_repo_snapshot
REPO_PATH = download_repo_snapshot('.', repo_name='electricdss-tst', use_version=False)
IEEE13_PATH = REPO_PATH / 'Version8/Distrib/IEEETestCases/13Bus/IEEE13Nodeckt.dss'
assert IEEE13_PATH.exists()

## The `Circuit.Save` function

It takes two arguments, a path for the file or directory to be used (if not saving to a string), and the options. It returns the DSS script as a string if `SingleFile|ToString` is used.

In [ ]:
from dss import DSSSaveFlags
from opendssdirect import dss as odd
from IPython.display import display, Markdown
from textwrap import dedent

In [ ]:
? odd.Circuit.Save

In [ ]:
display(Markdown(dedent(odd.Circuit.Save.__doc__)))

## Saving to a folder

This is equivalent to the `save circuit dir=save_dir`. Inspect the `save_dir` folder after running it.

In [ ]:
# Don't forget to load a circuit first:
odd(f'redirect "{IEEE13_PATH}"')

In [ ]:
print('Saving to', str(REPO_PATH / 'save_dir'))

In [ ]:
odd.Circuit.Save(str(REPO_PATH / 'save_dir'), 0)

## Saving to a single file

Saving to a single file can be useful to simplify copying the circuit. Sometimes the circuit is small enough that it can useful to keep everything in the same file.

Note that DSS-Extensions introduced a few lines always exported by default, since they are common sources of issues:

- A comment with the version of the engine, making it clear what was used and/or what is the target of the script. Although OpenDSS doesn't change frequently, it can break compatibility from time to time, and AltDSS/DSS C-API follows most of this kind of decision from the official OpenDSS (for compatibility and so on).
- It always sets the `DefaultBaseFreq`. New users frequently stumble on issues like loading a 50 Hz circuit on OpenDSS, followed by a 60 Hz circuit (which doesn't explicitly set this).
- Similarly, it always sets the `EarthModel`.

As suggested in the options, `CalcVoltageBases` is not included by default since many system require more careful handling of the base voltages than what `CalcVoltageBases` provides. If the user is sure that the circuit is fine with `CalcVoltageBases`, the option can be used or [the general compatibility flag](https://dss-extensions.org/OpenDSSDirect.py/enumerations.html#dss_python_backend.enums.DSSCompatFlags.SaveCalcVoltageBases) can be configured in [`Basic.CompatFlags()` function](https://dss-extensions.org/OpenDSSDirect.py/opendssdirect.html#opendssdirect.Basic.IBasic.CompatFlags).

In [ ]:
odd.Circuit.Save('saved_circuit.dss', DSSSaveFlags.SingleFile)
with open('saved_circuit.dss', 'r') as f:
    saved_data = f.read()

print(saved_data)

## Exporting directly to a string

One basic feature is to save circuits directly to a string, which is returned by the function. This is very useful for both inspecting and copying the data to another process.

Let's use the IEEE13 circuit since it's not too large. Even for this circuit, we can notice that a lot of the text is not user-provided data, but the default OpenDSS items (the basic items created automatically for every circuit).

In [ ]:
odd(f'redirect "{IEEE13_PATH}"')

In [ ]:
saved_data_from_str = odd.Circuit.Save('', DSSSaveFlags.SingleFile|DSSSaveFlags.ToString)

Besides the timestamp from the comment, the data is the save as when saved to the file:

In [ ]:
print(saved_data_from_str)

## Supressing defaults

Since most circuits do not actually use most of the default items, suppressing them can remove some noise from the output.

Note that if a default object is edited, they will be 

In [ ]:
print(odd.Circuit.Save('', DSSSaveFlags.SingleFile|DSSSaveFlags.ToString|DSSSaveFlags.ExcludeDefault))

## Keeping the order of elements

Some circuits, probably somewhat ill-conditioned, are very sensible to the order of elements. This means that saving the circuit and reloading it can give slightly different results. This is expected, given how the numerical methods used in the engine work, but it is understandable it the user needs to keep the save results. This is used, for example, to test some of the DSS-Extensions functions related to JSON imports/exports.

In [ ]:
print(odd.Circuit.Save('', DSSSaveFlags.SingleFile|DSSSaveFlags.ToString|DSSSaveFlags.ExcludeDefault|DSSSaveFlags.KeepOrder))

## Open elements

Keeping the order is generally enough to reproduce the save `SystemY` matrix, but there are also situations that some elements are normal-open.

In [ ]:
odd(f'''
    redirect "{IEEE13_PATH}"
    open Line.671692 terminal=1
''')

In [ ]:
print(odd.Circuit.Save('', DSSSaveFlags.SingleFile|DSSSaveFlags.ToString|DSSSaveFlags.ExcludeDefault|DSSSaveFlags.KeepOrder|DSSSaveFlags.IsOpen))

## Including the options

Finally, including the circuit options might be important to reproduce it:

In [ ]:
print(odd.Circuit.Save('', DSSSaveFlags.SingleFile|DSSSaveFlags.ToString|DSSSaveFlags.ExcludeDefault|DSSSaveFlags.KeepOrder|DSSSaveFlags.IsOpen|DSSSaveFlags.IncludeOptions))

## Interaction with `CompatFlags()`

Besides the `CalcVoltageBases` mentioned before, the `PropertyTracking` flag also affects some output. Property tracking was introduced as an attempt to make the saved data (both DSS scripts and JSON) less ambiguous.

In [ ]:
IEEE4_DSS = '''
clear
Set DefaultBaseFrequency=60
new circuit.4busDYBal   basekV=12.47 phases=3 mvasc3=200000 200000
set earthmodel=carson
new wiredata.conductor Runits=mi Rac=0.306 GMRunits=ft GMRac=0.0244  Radunits=in Diam=0.721 normamps=530
new wiredata.neutral   Runits=mi Rac=0.592 GMRunits=ft GMRac=0.00814 Radunits=in Diam=0.563 normamps=340
new linegeometry.4wire nconds=4 nphases=3 reduce=yes 
~ cond=1 wire=conductor units=ft x=-4   h=28 
~ cond=2 wire=conductor units=ft x=-1.5 h=28 
~ cond=3 wire=conductor units=ft x=3    h=28 
~ cond=4 wire=neutral   units=ft x=0    h=24 
new line.line1 geometry=4wire length=2000 units=ft bus1=sourcebus bus2=n2
new transformer.t1 xhl=6
~ wdg=1 bus=n2 conn=delta kV=12.47 kVA=6000 %r=0.5 
~ wdg=2 bus=n3 conn=wye   kV=4.16  kVA=6000 %r=0.5 
new line.line2 bus1=n3 bus2=n4 geometry=4wire length=2500 units=ft  
new load.load1 phases=3 bus1=n4 conn=wye kV=4.16 kW=5400 pf=0.9  model=1 vminpu=0.75
set voltagebases=[12.47, 4.16] 
'''

### Without property tracking

Note here that when the load is edited, it still outputs `kW`, `PF` and `kvar`, which can be misleading and could confuse third-party parsers that don't fully implement the OpenDSS parser:

In [ ]:
from dss import DSSCompatFlags
odd.Basic.CompatFlags(DSSCompatFlags.NoPropertyTracking)
odd(IEEE4_DSS)
odd('load.load1.kvar=1000')
print(odd.Circuit.Save('', DSSSaveFlags.SingleFile|DSSSaveFlags.ToString|DSSSaveFlags.ExcludeDefault|DSSSaveFlags.KeepOrder))

### With property tracking (default)

With property tracking, `PF` is supressed from the load, since the load definition switches from (`kW`, `PF`) to (`kW`, `kvar`).

Property tracking is an added functionaly on DSS-Extensions, the default OpenDSS does not implement this.

In [ ]:
odd.Basic.CompatFlags(0)
odd(IEEE4_DSS)
odd('load.load1.kvar=1000')
print(odd.Circuit.Save('', DSSSaveFlags.SingleFile|DSSSaveFlags.ToString|DSSSaveFlags.ExcludeDefault|DSSSaveFlags.KeepOrder))

## Is JSON a better alternative?

If the exported circuit will be used for other software, the engine in DSS-Extensions include work-in-progress implementation of JSON export and import functions. The repository for general discussions on this is at https://github.com/dss-extensions/AltDSS-Schema and the docs are also being updated.

For short, there are some small changes to match JSON expectations (this is encoded in the JSON Schema), some DSS properties (like winding and conductor iterators) are suppressed, etc.

Currently, some options and commands are included in the `PreCommands` and `PostCommands` lists. They will be formalized as JSON objects later.

In [ ]:
import json
json_data = json.loads(odd.Circuit.ToJSON())
json_data

In [ ]:
# If you are running on Jupyter, better display
from IPython.display import JSON
display(JSON(json_data))